In [82]:
using LinearAlgebra # do operacji macierzowych i testowania
using RandomNumbers.MersenneTwisters # do MT19937()
using Printf
using DelimitedFiles

epsilon = 10e-8;

In [83]:
# test data
function Ab_read_from_file(path)    
    local D1 = (open(readdlm, path))
    local T = []
    for i=1:50
        T = [T;(M =(A=convert(Array{Float64,2}, D1[(i-1)*100 + i+1:i*100+i, 1:end-1]), b=convert(Array{Float64,1}, D1[(i-1)*100 + i+1:i*100+i, end])), desc ="Big 100x100 "*string(i))]
    end
    return T
    end;

In [84]:
# test na macierzach. 
# f: funkcja zwracającą wynik do przetestowania z testem
# name: nazwa metody liczącej rząd macierzy
# rng: generator liczb losowych. Można przekazać np. MT19937(<seed>) aby generować te same zestawy losowych danych
# exact_f: funkcja do liczenia dokładnego wyniku, chyba że test zawiera pole `exact`, wtedy to ono jest użyte
# equal_f: opcjonalne do porównywania wartości wyliczonej przez f() i exact_f() lub test.exact_f(). Domyślnie: `==`
function test(f, f_name, exact_f; rng=nothing, equal_f=(x, y)->x==y)
    local _rand = rng==nothing ? rand : (args...)->rand(rng, args...) # custom rand if generator is set
      
    local Id_100x100          = (b=_rand(-10.0:10.0, 100), A=Matrix{Float64}(I, 100, 100))
    local small_3x3           = (A=[1.0 2.0 3.0; 4.0 5.0 6.0; 7.0 8.0 10.0], b=[10.0 12.0 13.0]')
    local big_100x100         = Ab_read_from_file("gauss1.in")
    # tests consist of fieds M=matrix, desc=description of test and exact=wanted
    tests = [
        (M=Id_100x100,          desc="Identity 100x100 matrix with random b")
        (M=small_3x3 ,          desc="Small hardcoded 3x3 test");
        big_100x100
    ];
    
    @printf(">>> Random test ID: [\e[1;35m%.16f\e[m]\n", _rand())

    function _do_test(test)
        local t1 = @elapsed exact = haskey(test, :exact_f) ? test.exact_f(test.M) : exact_f(test.M)
        local t2 = @elapsed computed = f(test.M) 
        local success = equal_f(exact, computed)
         @printf("[%5s] [exact:\ttime:%f] [%s:\ttime:%f] <<< %s\n", 
                 success ? "\e[1;32mPASS\e[m" : "\e[1;31mFAIL\e[m", t1, f_name, t2, test.desc)
    end

    @printf("> Hardcoded tests:\n")
    for test in tests
        _do_test(test)
    end
    
    flush(stdout)
    @printf(">>> TESTS END\n")
end

function getPrecisionU(T=Float64) return 0.5*2.0^-precision(T) end
# zamienia liczby zmiennoprzecinkowe w macierzy na zera jeśli |x|≦|maxError|
function filterZeros(matrix, maxError=epsilon) map(x->abs(x)<=abs(maxError) ? 0.0 : x, matrix) end
;

In [85]:
# solves Ax=b where Aₙₓₙ matrix of coefficients, bₙₓ₁ matrix of results for linear equations
function solve!(A, b)
    local M = [A b]'
    local n = length(M[1, :])
    local isZero = x->abs(x)<=epsilon
    # Gauss reduce on columns
    for col in 1:n
        # swap-in column with nonzero value
        local allzero = true
        for j in col:n 
            if (!isZero(M[col, j])) 
                M[:, col], M[:, j], allzero = M[:, j], M[:, col], false
                break
            end
        end
        if (allzero) continue end # skip to next step iff row with zeros till the end
        
        # gauss reduce columns
        for next_col in col+1:n
            M[:, next_col] -= M[:, col]*M[col, next_col]/M[col, col] # denom !=0 cuz we swapped
        end
    end
    
    for col in n:-1:1
        if (isZero(M[col, col]))
            throw(:UNSOLVABLE)
#             if (!isZero(M[end, col])) return NaN*ones(length(b)) end # no solutions fuz we have 0*x = y (!=0)
#             return Inf*ones(length(b)) # inf solutions cuz we have 0*x = 0
        end
        M[:, col] /= M[col, col] # calculate b[col]
        for j in col-1:-1:1
            M[:, j] -= M[col, j]*M[:, col]
        end
    end
    return M[end, :]
    
end
;

In [88]:
test(M->try solve!(M.A, M.b) catch; :UNSOLVABLE end, "solve!",
    M->try \(M.A, M.b) catch; :UNSOLVABLE end;
    equal_f=(v,w)->(v==w==:UNSOLVABLE || abs(norm(v-w))<=epsilon))

>>> Random test ID: [0.9610688406006067]
> Hardcoded tests:
[PASS] [exact:	time:0.000024] [solve!:	time:0.077138] <<< Identity 100x100 matrix with random b
[PASS] [exact:	time:0.000013] [solve!:	time:0.000007] <<< Small hardcoded 3x3 test
[PASS] [exact:	time:0.029928] [solve!:	time:0.014657] <<< Big 100x100 1
[PASS] [exact:	time:0.009936] [solve!:	time:0.016688] <<< Big 100x100 2
[PASS] [exact:	time:0.028179] [solve!:	time:0.014899] <<< Big 100x100 3
[PASS] [exact:	time:0.001478] [solve!:	time:0.016826] <<< Big 100x100 4
[PASS] [exact:	time:0.004257] [solve!:	time:0.017867] <<< Big 100x100 5
[PASS] [exact:	time:0.008792] [solve!:	time:0.014898] <<< Big 100x100 6
[PASS] [exact:	time:0.038288] [solve!:	time:0.012812] <<< Big 100x100 7
[PASS] [exact:	time:0.013450] [solve!:	time:0.017904] <<< Big 100x100 8
[PASS] [exact:	time:0.001066] [solve!:	time:0.017286] <<< Big 100x100 9
[PASS] [exact:	time:0.007394] [solve!:	time:0.017833] <<< Big 100x100 10
[PASS] [exact:	time:0.031534] [solve!:	t